In [ ]:
import os
from datasets import load_dataset, Audio
from transformers import AutoFeatureExtractor
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def warmup_cosine(
    warmup_steps, max_lr, total_steps, optimizer_lr=None, initial_lr=1e-6, min_lr=0
) -> Callable:
    """

    How to use it:
    import torch
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, warmup_cosine(...))
    :param warmup_steps:
    :param max_lr:
    :param total_steps:
    :param optimizer_lr: While using huggingface trainer, the output lr is multiplied to the main lr of the optimizer
    which is a bit confusing but if using huggingface and want to make sure to have exact values pass in the optimizer_lr
    value as well so that it will become uneffected.
    :param initial_lr:
    :param min_lr:
    :return:
    >>> epochs = 25
    >>> warmup_steps = epochs//10
    >>> max_lr = 5e-5
    >>> lambda_func = warmup_cosine(warmup_steps, max_lr, epochs)
    >>> for step in range(1, epochs + 1):
    ...     print("lr:", round(lambda_func(step), 6))
    lr: 1.5e-05
    lr: 5e-05
    lr: 5e-05
    lr: 5e-05
    lr: 4.9e-05
    lr: 4.8e-05
    lr: 4.7e-05
    lr: 4.6e-05
    lr: 4.4e-05
    lr: 4.3e-05
    lr: 4.1e-05
    lr: 3.9e-05
    lr: 3.7e-05
    lr: 3.4e-05
    lr: 3.2e-05
    lr: 2.9e-05
    lr: 2.6e-05
    lr: 2.3e-05
    lr: 2e-05
    lr: 1.7e-05
    lr: 1.3e-05
    lr: 1e-05
    lr: 7e-06
    lr: 3e-06
    lr: 0.0
    """
    assert initial_lr < max_lr, "max_lr can't be equal to or less than initial lr"
    assert warmup_steps < total_steps, "warmup_steps can't be larger than total_steps"

    def lambda_func(x):
        if x <= warmup_steps:
            lr = initial_lr + (1 - cosine_reduce(x, warmup_steps)) * (
                max_lr - initial_lr
            )
        else:
            x = x - warmup_steps
            lr = min_lr + cosine_reduce(x, total_steps - warmup_steps) * (
                max_lr - min_lr
            )
            lr = max(min_lr, lr)
        if optimizer_lr is not None and isinstance(optimizer_lr, float):
            return lr / optimizer_lr
        return lr

    return lambda_func

In [ ]:
def dump_pickle(file_path: str, file, mode: str = "wb"):
    import pickle

    with open(file_path, mode=mode) as f:
        pickle.dump(file, f)


def load_pickle(file_path: str, mode: str = "rb", encoding=""):
    import pickle

    with open(file_path, mode=mode) as f:
        return pickle.load(f, encoding=encoding)


In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")
feature_extractor

/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [ ]:
import pandas as pd
train = pd.read_csv('../data/cv-valid-train.csv')
test = pd.read_csv('../data/cv-valid-test.csv')

In [ ]:
train = train.drop(['text', 'up_votes', 'down_votes', 'age', 'accent', 'duration'], axis=1)
train = train[train.gender != 'other']
train = train.rename(columns={'filename':'audio_path', 'gender':'label'})
train = train.dropna()
test = test.drop(['text', 'up_votes', 'down_votes', 'age', 'accent', 'duration'], axis=1)
test = test[test.gender != 'other']
test = test.rename(columns={'filename':'audio_path', 'gender':'label'})
test = test.dropna()

In [ ]:
x = len(train) - 2 * len(train[train.label == 'female'])
train = train.drop(train[train['label'] == 'male'].sample(x).index)

In [ ]:
x = len(test) - 2 * len(test[test.label == 'female'])
test = test.drop(test[test['label'] == 'male'].sample(x).index)

In [ ]:
test.to_csv('../data/processed_test.csv')
train.to_csv('../data/processed_train.csv')

In [ ]:
train_path = "../data/processed_train.csv"
test_path = '../data/processed_test.csv'
dataset = load_dataset('csv', data_files={'train': train_path,
                                          'test': test_path})
dataset = dataset.cast_column("audio_path", Audio(sampling_rate=16_000))
dataset["train"][0]

Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 2295.10it/s]


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ba4f4949eb2ec7fd/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 1582.46it/s]


{'Unnamed: 0': 5,
 'audio_path': {'path': 'cv-valid-train/sample-000005.mp3',
  'array': array([ 5.49967413e-15, -7.54565261e-15,  4.40981961e-15, ...,
         -2.54557634e-08,  3.04820867e-08,  2.05002413e-08]),
  'sampling_rate': 16000},
 'label': 'female'}

In [ ]:
import random
import IPython.display as ipd
import librosa
index = random.randint(0, len(dataset['train']))

path = dataset['train'][index]['audio_path']['path']
waveform, sr = librosa.load(path)
text = dataset['train'][index]['label']
print(text)
ipd.Audio(waveform, rate=sr, autoplay=True)

female


In [ ]:
labels = set(dataset["train"]['label'])
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
label2id

{'female': '0', 'male': '1'}

In [ ]:
# save label2id to be used in test
os.makedirs("results/best", exist_ok=True)
dump_pickle("results/best/label2id.pkl", label2id)

In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio_path"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    label = [int(label2id[x]) for x in examples["label"]]
    inputs["label"] = label
    return inputs

In [ ]:
encoded_dataset = dataset.map(preprocess_function, remove_columns="audio_path", batched=True)
encoded_dataset['train'][0]

Map:  55%|█████▍    | 20000/36498 [12:49<10:25, 26.38 examples/s]/usr/local/lib/python3.8/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


{'Unnamed: 0': 5,
 'label': 0,
 'input_values': [0.0002895552897825837,
  0.0002895552897825837,
  0.0002895552897825837,
  0.0002895552897825837,
  0.0002895552897825837,
  0.0002895552897825837,
  0.0002895552897825837,
  0.0002895552897825837,
  0.00028955531888641417,
  0.0002895552897825837,
  0.0002895552897825837,
  0.0002895552897825837,
  0.0002895552897825837,
  0.00028955531888641417,
  0.0002895552315749228,
  0.0002895552897825837,
  0.0002895552897825837,
  0.0002895552897825837,
  0.00028955514426343143,
  0.0002895552897825837,
  0.00028955526067875326,
  0.000289555115159601,
  0.0002895552897825837,
  0.0002895552315749228,
  0.0002895554935093969,
  0.00028955540619790554,
  0.0002895553479902446,
  0.0002895550278481096,
  0.00028955531888641417,
  0.0002895552897825837,
  0.0002895550860557705,
  0.000289555115159601,
  0.00028955464949831367,
  0.0002895547659136355,
  0.0002895552315749228,
  0.00028955412562936544,
  0.0002895543002523482,
  0.000289555144263431

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    recall = recall_score(labels, predictions, average="weighted")
    precision = precision_score(labels, predictions, average="weighted")

    return {"accuracy": acc, "f1-score": f1, "recall-score": recall, "precision-score": precision}

## TRAIN

In [ ]:
import math
import torch
from transformers import EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(early_stopping_patience=5)

train_bs = 64 
epochs = 25
lr = 5e-5
lrf = lr
output_dir = "./results"
total_steps = int((np.ceil(encoded_dataset["train"].num_rows / train_bs) * epochs))

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=epochs,
    report_to="tensorboard",
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model='loss',
    per_device_train_batch_size = train_bs,
    per_device_eval_batch_size = 64,
    logging_steps=1,
)

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, warmup_cosine(100,
                                                                       max_lr=lr,
                                                                       total_steps=total_steps,
                                                                       optimizer_lr=lr,
                                                                       min_lr=1e-6))
# reduce lr with a cosine annealing if total_steps is set to total_steps
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler)
)

trainer.train()
trainer.save_model(os.path.join(output_dir, "best"))

/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceCl

Epoch,Training Loss,Validation Loss


In [ ]:
torch.save(model, '/data/audio-classification-pytorch/wav2vec2/results/model1.pt')

In [ ]:
dump_pickle("results/best/id2label.pkl", id2label)

In [ ]:
label2id = load_pickle('/data/audio-classification-pytorch/wav2vec2/results/best/label2id.pkl')

In [ ]:
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=len(label2id), label2id=label2id, id2label=id2label
)

In [ ]:
checkpoint = torch.load('/data/audio-classification-pytorch/wav2vec2/results/best/pytorch_model.bin')

In [ ]:
model.load_state_dict(checkpoint)

## TEST

In [ ]:
import torchaudio
import torch
import librosa
device = "cpu"
model = model.to(device)
waveform, sr = librosa.load("../audio_samples/man_02.mp4")
waveform = torch.from_numpy(waveform).unsqueeze(0)
waveform = torchaudio.transforms.Resample(sr, 16_000)(waveform)
inputs = feature_extractor(waveform, sampling_rate=feature_extractor.sampling_rate,
                           max_length=16000, truncation=True)
tensor = torch.tensor(inputs['input_values'][0]).to(device)
with torch.no_grad():
    output = model(tensor)
    logits = output['logits'][0]
    label_id = torch.argmax(logits).item()
label_name = id2label[str(label_id)]
print(label_name)

OSError: /usr/local/lib/python3.8/dist-packages/torchaudio/lib/libtorchaudio.so: undefined symbol: _ZNK5torch8autograd4Node4nameEv

In [ ]:
import torchaudio
import torch
import librosa
device = "cpu"
model = model.to(device)
correct = 0
for i, test in enumerate(dataset['test']):
    path = test['audio_path']['path']
    waveform, sr = librosa.load(path)
    waveform = torch.from_numpy(waveform).unsqueeze(0)
    waveform = torchaudio.transforms.Resample(sr, 16_000)(waveform)
    inputs = feature_extractor(waveform, sampling_rate=feature_extractor.sampling_rate,
                            max_length=16000, truncation=True)
    tensor = torch.tensor(inputs['input_values'][0]).to(device)
    with torch.no_grad():
        output = model(tensor)
        logits = output['logits'][0]
        label_id = torch.argmax(logits).item()
    label_name = id2label[str(label_id)]
    if label_name == test['label']:
        correct += 1

In [ ]:
correct / len(dataset['test'])

In [ ]:
dump_pickle('/data/audio-classification-pytorch/wav2vec2/results', model)

IsADirectoryError: [Errno 21] Is a directory: '/data/audio-classification-pytorch/wav2vec2/results'

In [110]:
import torchaudio

OSError: /usr/local/lib/python3.8/dist-packages/torchaudio/lib/libtorchaudio.so: undefined symbol: _ZNK5torch8autograd4Node4nameEv